In [1]:
import jsonl_io
from tqdm import tqdm
import re
import json
from nltk import word_tokenize
import os

In [2]:
os.makedirs('../dataset/metadata/', exist_ok=True)
os.makedirs('../dataset/all/', exist_ok=True)

In [3]:
def offset_to_conll(text, labels):
    ents = []
    prev_idx = 0
    masked_text = ""
    for start, end, label in labels:
        prev_text = text[prev_idx:start]
        masked_text += prev_text
        masked_text += 'ENT'
        ent_text = text[start:end]
        tokenized_ent = word_tokenize(ent_text, language='portuguese', preserve_line=True)
        prev_idx = end
        iob_ents = []
        for idx, tok in enumerate(tokenized_ent):
            if not idx:
                iob_ents.append(f'{tok} B-{label}')
            else:
                iob_ents.append(f'{tok} I-{label}')
        ents.append(iob_ents)
    last_chunk = text[prev_idx:]
    masked_text += last_chunk
    
    final_text = ""
    
    ent_idx = 0
    for line in masked_text.splitlines():
        line_tokens = word_tokenize(line, language='portuguese', preserve_line=True)
        for token in line_tokens:
            if token == 'ENT':
                final_text += '\n'.join(ents[ent_idx])
                final_text += '\n'
                ent_idx += 1
            else:
                final_text += f'{token} O\n'
        final_text += '\n'
    
    return final_text

In [4]:
annotated_data = jsonl_io.read_jsonl('../processed.jsonl')

In [5]:
for doc in tqdm(annotated_data):
    n_sei = doc['n_sei']
    metadata = {}
    metadata['numero_sei'] = n_sei
    metadata['origem'] = doc['origem']
    metadata['tipo_documento'] = doc['tipo_documento']
    metadata_filename = f'{n_sei}_meta.json'
    conll_filename = f'{n_sei}.conll'
    with open('../dataset/metadata/' + metadata_filename, 'w') as f:
        json.dump(metadata, f)
    text = doc['data']
    labels = sorted(doc['label'])
    conll = offset_to_conll(text, labels)
    with open('../dataset/all/' + conll_filename, 'w') as f:
        f.write(conll)   

100%|██████████| 100/100 [00:02<00:00, 35.29it/s]
